In [1]:
// Официальная документация: https://jtablesaw.github.io/tablesaw/
// Пример с деревом: https://jtablesaw.github.io/tablesaw/userguide/ml/Heart%20Random%20forest
// Хороший блог для изучения: https://self-learning-java-tutorial.blogspot.com/search/label/TableSaw

In [8]:
%%loadFromPOM
<dependency>
  <groupId>tech.tablesaw</groupId>
  <artifactId>tablesaw-core</artifactId>
  <version>0.41.0</version>
</dependency>
<dependency>
  <groupId>tech.tablesaw</groupId>
  <artifactId>tablesaw-json</artifactId>
  <version>0.41.0</version>
</dependency>
<dependency>
    <groupId>commons-io</groupId>
    <artifactId>commons-io</artifactId>
    <version>2.11.0</version>
</dependency>
<dependency>
  <groupId>com.github.haifengl</groupId>
  <artifactId>smile-core</artifactId>
  <version>2.5.0</version>
</dependency>
<dependency>
    <groupId>tech.tablesaw</groupId>
    <artifactId>tablesaw-jsplot</artifactId>
    <version>0.41.0</version>
</dependency>

In [9]:
// import tech.tablesaw.io.json.JsonReader;
// import tech.tablesaw.io.json.JsonWriter;
import java.util.Arrays;
import tech.tablesaw.api.IntColumn;
import tech.tablesaw.api.StringColumn;
import tech.tablesaw.api.NumberColumn;
import tech.tablesaw.api.Table;
import tech.tablesaw.io.Source;
import smile.classification.RandomForest;
import smile.classification.DecisionTree;
import smile.base.cart.SplitRule;
// import smile.validation.metric.Accuracy;
import smile.validation.Accuracy;
import tech.tablesaw.plotly.Plot;
import tech.tablesaw.plotly.api.HorizontalBarPlot;
import tech.tablesaw.plotly.api.LinePlot;
import tech.tablesaw.plotly.components.Figure;
import tech.tablesaw.plotly.components.Page;
import smile.data.formula.Formula;
import tech.tablesaw.api.DoubleColumn;
import tech.tablesaw.api.StringColumn;
import tech.tablesaw.api.IntColumn;
import tech.tablesaw.columns.numbers.NumberColumnFormatter;
import tech.tablesaw.api.ColumnType;
import java.io.*;
// import tech.tablesaw.plotly.api.*;
// import tech.tablesaw.plotly.components.*;
// import tech.tablesaw.plotly.traces.*;

In [10]:
Table df = Table.read().file("titanic.csv");
df.first(3).print();

                                                                                                                                                                     
 Survived  |  Pclass  |                         Name                         |   Sex    |  Age  |  Siblings/Spouses Aboard  |  Parents/Children Aboard  |   Fare    |
---------------------------------------------------------------------------------------------------------------------------------------------------------------------
        0  |       3  |                              Mr. Owen Harris Braund  |    male  |   22  |                        1  |                        0  |     7.25  |
        1  |       1  |  Mrs. John Bradley (Florence Briggs Thayer) Cumings  |  female  |   38  |                        1  |                        0  |  71.2833  |
        1  |       3  |                               Miss. Laina Heikkinen  |  female  |   26  |                        0  |                        0  |    7.925  |

In [11]:
df.structure();

                    Structure of                     
 Index  |        Column Name        |  Column Type  |
-----------------------------------------------------
     0  |                 Survived  |      INTEGER  |
     1  |                   Pclass  |      INTEGER  |
     2  |                     Name  |       STRING  |
     3  |                      Sex  |       STRING  |
     4  |                      Age  |       DOUBLE  |
     5  |  Siblings/Spouses Aboard  |      INTEGER  |
     6  |  Parents/Children Aboard  |      INTEGER  |
     7  |                     Fare  |       DOUBLE  |

In [12]:
Table df = Table.read().file("titanic.csv");

// Перевести объектные колонки в integers. (преобразование onehot пока не нашёл)
df.replaceColumn("Sex", df.stringColumn("Sex").asDoubleColumn().asIntColumn());

// Удалить неважные колонки
df = df.removeColumns("Name");

// Удалить строки с пустыми ячейками 
df = df.dropRowsWithMissingValues();

// разделяем данные на тренировочную и проверочную выборку
Table[] splitData = df.sampleSplit(0.7);
Table df_Train = splitData[0];
Table df_Test = splitData[1];

// initial model with sensible parameters
RandomForest RFModel1 = smile.classification.RandomForest.fit(
     Formula.lhs("Survived"),
     df_Train.smile().toDataFrame(),
     50, //n
     (int) Math.sqrt((double) (df_Train.columnCount() - 1)), //m = sqrt(p)
     SplitRule.GINI,
     7, // d 
     100, // maxNodes
     1,
     1
);

// посмотрим на дерево решений для предсказаний
System.out.println(RFModel1.trees()[0]);

n=621
node), split, n, loss, yval, (yprob)
* denotes terminal node
1) root 621 822.80 0 (0.62279 0.37721)
 2) Age<=16.5000 84 114.73 1 (0.43023 0.56977)
  4) Pclass<=2.50000 20 8.4176 1 (0.090909 0.90909) *
  5) Pclass>2.50000 64 88.160 0 (0.54545 0.45455)
   10) Fare<=20.1666 28 19.244 1 (0.13333 0.86667)
    20) Fare<=16.9500 23 8.7202 1 (0.080000 0.92000) *
    21) Fare>16.9500 5 6.7469 1 (0.42857 0.57143)
     42) Fare<=19.0229 2 1.1507 0 (0.75000 0.25000) *
     43) Fare>19.0229 3 1.3389 1 (0.20000 0.80000) *
   11) Fare>20.1666 36 25.254 0 (0.86842 0.13158)
    22) Siblings/Spouses Aboard<=0.500000 4 4.6300 1 (0.33333 0.66667) *
    23) Siblings/Spouses Aboard>0.500000 32 9.4252 0 (0.94118 0.058824) *
 3) Age>16.5000 537 692.91 0 (0.65306 0.34694)
  6) Fare<=20.2313 318 332.67 0 (0.78125 0.21875)
   12) Sex<=0.500000 238 159.97 0 (0.89167 0.10833) *
   13) Sex>0.500000 80 110.10 1 (0.45122 0.54878)
    26) Fare<=17.2500 77 105.17 1 (0.43038 0.56962)
     52) Age<=56.0000 71 94.32

In [13]:
// сделаем предсказание на тестовой выборке и оценим точность работы модели
int[] predictions = RFModel1.predict(df_Test.smile().toDataFrame());
double accuracy1 = Accuracy.of(df_Test.intColumn("Survived").asIntArray(), predictions);
System.out.println(accuracy1);

0.8120300751879699


In [14]:
// найдём важность параметров, которые влияют на результат
double[] RFModel1_Importance = RFModel1.importance();

Table varImportance = Table.create("featureImportance");

List<String> featureNames = df_Train.columnNames();

featureNames.remove("Survived");


// построим таблицу
varImportance.addColumns(DoubleColumn.create("featureImportance", RFModel1_Importance), StringColumn.create("Feature",  featureNames));
varImportance = varImportance.sortAscendingOn("featureImportance");
varImportance

                featureImportance                 
 featureImportance   |          Feature          |
--------------------------------------------------
  4.276720520175648  |                   Pclass  |
  4.932680628637136  |  Parents/Children Aboard  |
  16.84819799398992  |  Siblings/Spouses Aboard  |
  17.46714320336007  |                      Sex  |
 33.082348465017475  |                      Age  |
  33.73461616864792  |                     Fare  |

In [15]:
// сделаем график из этой таблицы

// Примеры построения графиков: https://github.com/jtablesaw/tablesaw/tree/master/jsplot/src/test/java/tech/tablesaw/examples
Figure figure = HorizontalBarPlot.create("Feature Importance", varImportance, "Feature", "featureImportance");

// график будем строить на странице JupyterLab, поэтому исопльзуем iframe:
String raw = Page.pageBuilder(figure, "plot_div_id").build().toString();
raw = "data:text/html;charset=utf-8;base64," + Base64.getEncoder().encodeToString(raw.getBytes("ascii"));
render("<iframe width=\"100%\" height=\"700\" src=\"" + raw + "\"></iframe>", "text/html");

<iframe width="100%" height="700" src="data:text/html;charset=utf-8;base64,PCFET0NUWVBFIGh0bWw+CjxodG1sIGxhbmc9ImVuLVVTIj4KPGhlYWQ+CiAgICA8bWV0YSBjaGFyc2V0PSJVVEYtOCI+CiAgICA8dGl0bGU+RmVhdHVyZSBJbXBvcnRhbmNlPC90aXRsZT4KICAgIDxzY3JpcHQgc3JjPSJodHRwczovL2Nkbi5wbG90Lmx5L3Bsb3RseS1sYXRlc3QubWluLmpzIj48L3NjcmlwdD4KPC9oZWFkPgo8Ym9keT4KICAgIDxkaXYgaWQ9J3Bsb3RfZGl2X2lkJyA+PC9kaXY+CiAgICAgICAgPHNjcmlwdD4KICAgICAgICB2YXIgdGFyZ2V0X3Bsb3RfZGl2X2lkID0gZG9jdW1lbnQuZ2V0RWxlbWVudEJ5SWQoJ3Bsb3RfZGl2X2lkJyk7CiAgICAgICAgdmFyIGxheW91dCA9IHsKICAgIHRpdGxlOiAnRmVhdHVyZSBJbXBvcnRhbmNlJywKICAgIGhlaWdodDogNzAwLAogICAgd2lkdGg6IDkwMCwKCgp9OwoKdmFyIHRyYWNlMCA9CnsKeDogWzQuMjc2NzIwNTIwMTc1NjQ4LCA0LjkzMjY4MDYyODYzNzEzNiwgMTYuODQ4MTk3OTkzOTg5OTIsIDE3LjQ2NzE0MzIwMzM2MDA3LCAzMy4wODIzNDg0NjUwMTc0NzUsIDMzLjczNDYxNjE2ODY0NzkyXSwKeTogWyJQY2xhc3MiLCJQYXJlbnRzL0NoaWxkcmVuIEFib2FyZCIsIlNpYmxpbmdzL1Nwb3VzZXMgQWJvYXJkIiwiU2V4IiwiQWdlIiwiRmFyZSJdLApvcmllbnRhdGlvbjogJ2gnLAp4YXhpczogJ3gnLAp5YXhpczogJ3knLAp0eXBlOiAnYmFyJywKbmFtZTogJycsCn07CgogICAgICAgIHZhciBkYXRhID0gWyB0cmFjZTBdOwpQbG90bHkubmV3UGxvdCh0YXJnZXRfcGxvdF9kaXZfaWQsIGRhdGEsIGxheW91dCk7ICAgICAgICAgICAgPC9zY3JpcHQ+Cgo8L2JvZHk+CjwvaHRtbD4="></iframe>

In [16]:
// Строим матрицу корреляции, чтобы убрать очень сильно и слабо зависимые параметры
Table corr = Table.create("Spearman's Correlation Matrix");

corr.addColumns(StringColumn.create("Feature"));

for(String name: df_Test.columnNames().subList(0, df_Test.columns().size())) {
    corr.addColumns(DoubleColumn.create(name));
}

for(int i = 0; i < df_Test.columns().size(); i++) {
    for(int j = 0; j < df_Test.columns().size(); j++) {
        corr.doubleColumn(i+1).append(df_Train.numericColumns(i).get(0).asDoubleColumn().spearmans(df_Train.numericColumns(j).get(0).asDoubleColumn()));
    }
}

corr.stringColumn("Feature").addAll(df_Train.columnNames().subList(0, df_Test.columns().size()));

for(Object ea: corr.columnsOfType(ColumnType.DOUBLE)) {
    ((NumberColumn) ea).setPrintFormatter(NumberColumnFormatter.fixedWithGrouping(2));
}

System.out.println(corr.printAll());

                                                      Spearman's Correlation Matrix                                                      
         Feature          |  Survived  |  Pclass  |   Sex   |   Age   |  Siblings/Spouses Aboard  |  Parents/Children Aboard  |  Fare   |
-----------------------------------------------------------------------------------------------------------------------------------------
                Survived  |      1.00  |   -0.31  |   0.55  |  -0.09  |                     0.07  |                     0.17  |   0.30  |
                  Pclass  |     -0.31  |    1.00  |  -0.10  |  -0.38  |                    -0.05  |                    -0.02  |  -0.69  |
                     Sex  |      0.55  |   -0.10  |   1.00  |  -0.12  |                     0.16  |                     0.27  |   0.22  |
                     Age  |     -0.09  |   -0.38  |  -0.12  |   1.00  |                    -0.19  |                    -0.29  |   0.14  |
 Siblings/Spouses Aboard  |      0

In [17]:
/*
// Так как во всех столбцах есть хорошие числа (больше 0.1), то не будем исбавляться от них. Но если захочется, то всегда может сделать так:

dataTest.removeColumns("Fbs");
dataTrain.removeColumns("Fbs");
*/

In [18]:
// Тюнингуем гиперпараметры модели (кол-во деревьев)
Table AccuracyvTrees = Table.create("OOB-Error vs nTrees");  // Out-of-bounds error
AccuracyvTrees.addColumns(DoubleColumn.create("OOBerror"), DoubleColumn.create("ntrees"));

for(int j = 50; j < 2000; j = j+10) {
    RandomForest model = smile.classification.RandomForest.fit(
        Formula.lhs("Survived"),
        df_Train.smile().toDataFrame(),
        j,
        (int) Math.sqrt((double) (df_Train.columnCount() - 1)), //root p
        SplitRule.GINI,
        7,
        100,
        1,
        1
    );

    double err = model.error();
    AccuracyvTrees.doubleColumn(0).append(err);
    AccuracyvTrees.doubleColumn(1).append(j);
}

Figure figure = LinePlot.create("Accuracy", AccuracyvTrees, "ntrees", "OOBerror");

String raw = Page.pageBuilder(figure, "plot_div_id").build().toString();
raw = "data:text/html;charset=utf-8;base64," + Base64.getEncoder().encodeToString(raw.getBytes("ascii"));
render("<iframe width=\"100%\" height=\"600\" src=\"" + raw + "\"></iframe>", "text/html");

<iframe width="100%" height="600" src="data:text/html;charset=utf-8;base64,PCFET0NUWVBFIGh0bWw+CjxodG1sIGxhbmc9ImVuLVVTIj4KPGhlYWQ+CiAgICA8bWV0YSBjaGFyc2V0PSJVVEYtOCI+CiAgICA8dGl0bGU+QWNjdXJhY3k8L3RpdGxlPgogICAgPHNjcmlwdCBzcmM9Imh0dHBzOi8vY2RuLnBsb3QubHkvcGxvdGx5LWxhdGVzdC5taW4uanMiPjwvc2NyaXB0Pgo8L2hlYWQ+Cjxib2R5PgogICAgPGRpdiBpZD0ncGxvdF9kaXZfaWQnID48L2Rpdj4KICAgICAgICA8c2NyaXB0PgogICAgICAgIHZhciB0YXJnZXRfcGxvdF9kaXZfaWQgPSBkb2N1bWVudC5nZXRFbGVtZW50QnlJZCgncGxvdF9kaXZfaWQnKTsKICAgICAgICB2YXIgbGF5b3V0ID0gewogICAgdGl0bGU6ICdBY2N1cmFjeScsCiAgICBoZWlnaHQ6IDYwMCwKICAgIHdpZHRoOiA4MDAsCiAgICB4YXhpczogewogICAgdGl0bGU6ICdudHJlZXMnLAogICAgfSwKCiAgICB5YXhpczogewogICAgdGl0bGU6ICdPT0JlcnJvcicsCiAgICB9LAoKfTsKCnZhciB0cmFjZTAgPQp7Cng6IFsiNTAuMCIsIjYwLjAiLCI3MC4wIiwiODAuMCIsIjkwLjAiLCIxMDAuMCIsIjExMC4wIiwiMTIwLjAiLCIxMzAuMCIsIjE0MC4wIiwiMTUwLjAiLCIxNjAuMCIsIjE3MC4wIiwiMTgwLjAiLCIxOTAuMCIsIjIwMC4wIiwiMjEwLjAiLCIyMjAuMCIsIjIzMC4wIiwiMjQwLjAiLCIyNTAuMCIsIjI2MC4wIiwiMjcwLjAiLCIyODAuMCIsIjI5MC4wIiwiMzAwLjAiLCIzMTAuMCIsIjMyMC4wIiwiMzMwLjAiLCIzNDAuMCIsIjM1MC4wIiwiMzYwLjAiLCIzNzAuMCIsIjM4MC4wIiwiMzkwLjAiLCI0MDAuMCIsIjQxMC4wIiwiNDIwLjAiLCI0MzAuMCIsIjQ0MC4wIiwiNDUwLjAiLCI0NjAuMCIsIjQ3MC4wIiwiNDgwLjAiLCI0OTAuMCIsIjUwMC4wIiwiNTEwLjAiLCI1MjAuMCIsIjUzMC4wIiwiNTQwLjAiLCI1NTAuMCIsIjU2MC4wIiwiNTcwLjAiLCI1ODAuMCIsIjU5MC4wIiwiNjAwLjAiLCI2MTAuMCIsIjYyMC4wIiwiNjMwLjAiLCI2NDAuMCIsIjY1MC4wIiwiNjYwLjAiLCI2NzAuMCIsIjY4MC4wIiwiNjkwLjAiLCI3MDAuMCIsIjcxMC4wIiwiNzIwLjAiLCI3MzAuMCIsIjc0MC4wIiwiNzUwLjAiLCI3NjAuMCIsIjc3MC4wIiwiNzgwLjAiLCI3OTAuMCIsIjgwMC4wIiwiODEwLjAiLCI4MjAuMCIsIjgzMC4wIiwiODQwLjAiLCI4NTAuMCIsIjg2MC4wIiwiODcwLjAiLCI4ODAuMCIsIjg5MC4wIiwiOTAwLjAiLCI5MTAuMCIsIjkyMC4wIiwiOTMwLjAiLCI5NDAuMCIsIjk1MC4wIiwiOTYwLjAiLCI5NzAuMCIsIjk4MC4wIiwiOTkwLjAiLCIxMDAwLjAiLCIxMDEwLjAiLCIxMDIwLjAiLCIxMDMwLjAiLCIxMDQwLjAiLCIxMDUwLjAiLCIxMDYwLjAiLCIxMDcwLjAiLCIxMDgwLjAiLCIxMDkwLjAiLCIxMTAwLjAiLCIxMTEwLjAiLCIxMTIwLjAiLCIxMTMwLjAiLCIxMTQwLjAiLCIxMTUwLjAiLCIxMTYwLjAiLCIxMTcwLjAiLCIxMTgwLjAiLCIxMTkwLjAiLCIxMjAwLjAiLCIxMjEwLjAiLCIxMjIwLjAiLCIxMjMwLjAiLCIxMjQwLjAiLCIxMjUwLjAiLCIxMjYwLjAiLCIxMjcwLjAiLCIxMjgwLjAiLCIxMjkwLjAiLCIxMzAwLjAiLCIxMzEwLjAiLCIxMzIwLjAiLCIxMzMwLjAiLCIxMzQwLjAiLCIxMzUwLjAiLCIxMzYwLjAiLCIxMzcwLjAiLCIxMzgwLjAiLCIxMzkwLjAiLCIxNDAwLjAiLCIxNDEwLjAiLCIxNDIwLjAiLCIxNDMwLjAiLCIxNDQwLjAiLCIxNDUwLjAiLCIxNDYwLjAiLCIxNDcwLjAiLCIxNDgwLjAiLCIxNDkwLjAiLCIxNTAwLjAiLCIxNTEwLjAiLCIxNTIwLjAiLCIxNTMwLjAiLCIxNTQwLjAiLCIxNTUwLjAiLCIxNTYwLjAiLCIxNTcwLjAiLCIxNTgwLjAiLCIxNTkwLjAiLCIxNjAwLjAiLCIxNjEwLjAiLCIxNjIwLjAiLCIxNjMwLjAiLCIxNjQwLjAiLCIxNjUwLjAiLCIxNjYwLjAiLCIxNjcwLjAiLCIxNjgwLjAiLCIxNjkwLjAiLCIxNzAwLjAiLCIxNzEwLjAiLCIxNzIwLjAiLCIxNzMwLjAiLCIxNzQwLjAiLCIxNzUwLjAiLCIxNzYwLjAiLCIxNzcwLjAiLCIxNzgwLjAiLCIxNzkwLjAiLCIxODAwLjAiLCIxODEwLjAiLCIxODIwLjAiLCIxODMwLjAiLCIxODQwLjAiLCIxODUwLjAiLCIxODYwLjAiLCIxODcwLjAiLCIxODgwLjAiLCIxODkwLjAiLCIxOTAwLjAiLCIxOTEwLjAiLCIxOTIwLjAiLCIxOTMwLjAiLCIxOTQwLjAiLCIxOTUwLjAiLCIxOTYwLjAiLCIxOTcwLjAiLCIxOTgwLjAiLCIxOTkwLjAiXSwKeTogWyIwLjE3MDY5MjQzMTU2MTk5NjgiLCIwLjE3NzEzMzY1NTM5NDUyNDk3IiwiMC4xNjc0NzE4MTk2NDU3MzI2OCIsIjAuMTY0MjUxMjA3NzI5NDY4NiIsIjAuMTU5NDIwMjg5ODU1MDcyNDUiLCIwLjE2MTAzMDU5NTgxMzIwNDUiLCIwLjE2NTg2MTUxMzY4NzYwMDYzIiwiMC4xNzA2OTI0MzE1NjE5OTY4IiwiMC4xNTI5NzkwNjYwMjI1NDQzIiwiMC4xNjU4NjE1MTM2ODc2MDA2MyIsIjAuMTY0MjUxMjA3NzI5NDY4NiIsIjAuMTY1ODYxNTEzNjg3NjAwNjMiLCIwLjE2MjY0MDkwMTc3MTMzNjU1IiwiMC4xNjI2NDA5MDE3NzEzMzY1NSIsIjAuMTY0MjUxMjA3NzI5NDY4NiIsIjAuMTU5NDIwMjg5ODU1MDcyNDUiLCIwLjE2MjY0MDkwMTc3MTMzNjU1IiwiMC4xNjEwMzA1OTU4MTMyMDQ1IiwiMC4xNjEwMzA1OTU4MTMyMDQ1IiwiMC4xNjI2NDA5MDE3NzEzMzY1NSIsIjAuMTY1ODYxNTEzNjg3NjAwNjMiLCIwLjE2MjY0MDkwMTc3MTMzNjU1IiwiMC4xNjc0NzE4MTk2NDU3MzI2OCIsIjAuMTYyNjQwOTAxNzcxMzM2NTUiLCIwLjE2MjY0MDkwMTc3MTMzNjU1IiwiMC4xNjI2NDA5MDE3NzEzMzY1NSIsIjAuMTYyNjQwOTAxNzcxMzM2NTUiLCIwLjE2MjY0MDkwMTc3MTMzNjU1IiwiMC4xNTk0MjAyODk4NTUwNzI0NSIsIjAuMTYyNjQwOTAxNzcxMzM2NTUiLCIwLjE2NDI1MTIwNzcyOTQ2ODYiLCIwLjE2MjY0MDkwMTc3MTMzNjU1IiwiMC4xNjQyNTEyMDc3Mjk0Njg2IiwiMC4xNjU4NjE1MTM2ODc2MDA2MyIsIjAuMTY0MjUxMjA3NzI5NDY4NiIsIjAuMTYyNjQwOTAxNzcxMzM2NTUiLCIwLjE2OTA4M

In [19]:
// Минимум ошибок примерно на значении 325 деревьев. 
// Закончим исследование и проверим предсказательную силу лучшей модели.
RandomForest RFModelBest = smile.classification.RandomForest.fit(
    Formula.lhs("Survived"),
    df_Train.smile().toDataFrame(),
    150,
    (int) Math.sqrt((double) (df_Train.columnCount() - 1)), //root p
    SplitRule.GINI,
    7,
    100,
    1,
    1
);

int[] predictionsBest = RFModelBest.predict(df_Test.smile().toDataFrame());
double accuracyBest = Accuracy.of(df_Test.intColumn("Survived").asIntArray(), predictionsBest);
accuracyBest

0.8195488721804511

In [20]:
// Применим модель на практике, передав ей для предсказания один набор значений:

// Данные, которые вводит пользователь для получения предсказания
// int[] Survived = { Integer.parseInt("1") };
int[] Pclass = { Integer.parseInt("1") };
String[] Sex = { "male" };
// String[] Name = { "Name" };
double[] Age = { Double.parseDouble("13") };
int[] Siblings_Spouses_Aboard = { Integer.parseInt("1") };
int[] Parents_Children_Aboard = { Integer.parseInt("2") };
double[] Fare = { Double.parseDouble("71.2833") };

/* Должны получиться такие типы колонок в таблице
 Index  |        Column Name        |  Column Type  |
-----------------------------------------------------
     1  |                   Pclass  |      INTEGER  |
     2  |                      Sex  |       STRING  |
     3  |                      Age  |       DOUBLE  |
     4  |  Siblings/Spouses Aboard  |      INTEGER  |
     5  |  Parents/Children Aboard  |      INTEGER  |
     6  |                     Fare  |       DOUBLE  |
*/

Table tm = Table.create()
    // .addColumns(IntColumn.create("Survived", Survived))
    .addColumns(IntColumn.create("Pclass", Pclass))
    // .addColumns(StringColumn.create("Name", Name))
    .addColumns(StringColumn.create("Sex", Sex))
    .addColumns(DoubleColumn.create("Age", Age))
    .addColumns(IntColumn.create("Siblings/Spouses Aboard", Siblings_Spouses_Aboard))
    .addColumns(IntColumn.create("Parents/Children Aboard", Parents_Children_Aboard))
    .addColumns(DoubleColumn.create("Fare", Fare));

// делаем те же самые преобразование с датасетом, что и при тренировке
tm.replaceColumn("Sex", tm.stringColumn("Sex").asDoubleColumn().asIntColumn());
tm;

// Делаем предсказание
int[] prediction = RFModelBest.predict(tm.smile().toDataFrame());

int result = prediction[0];

result // 1 или 0

1

In [25]:
// записываем модель в файл
FileOutputStream fileOut = new FileOutputStream("./t.smile.model.ser");
ObjectOutputStream out = new ObjectOutputStream(fileOut);
out.writeObject(RFModelBest);
out.close();
fileOut.close();

In [13]:
// читаем модель из файла
RandomForest e = null;
FileInputStream fileIn = new FileInputStream("./t.smile.model.ser");
ObjectInputStream in = new ObjectInputStream(fileIn);
e = (RandomForest) in.readObject();
e

smile.classification.RandomForest@78b1a922

In [15]:
InputStream fileIn = this.getClass().getResourceAsStream("./t.smile.model.ser");
ObjectInputStream in = new ObjectInputStream(fileIn);

CompilationException: 

In [23]:
// Применим модель на практике, передав ей для предсказания один набор значений:

// Данные, которые вводит пользователь для получения предсказания
// int[] Survived = { Integer.parseInt("1") };
int[] Pclass = { Integer.parseInt("1") };
String[] Sex = { "female" };
// String[] Name = { "Name" };
double[] Age = { Double.parseDouble("4") };
int[] Siblings_Spouses_Aboard = { Integer.parseInt("1") };
int[] Parents_Children_Aboard = { Integer.parseInt("2") };
double[] Fare = { Double.parseDouble("150.2833") };

/* Должны получиться такие типы колонок в таблице
 Index  |        Column Name        |  Column Type  |
-----------------------------------------------------
     1  |                   Pclass  |      INTEGER  |
     2  |                      Sex  |       STRING  |
     3  |                      Age  |       DOUBLE  |
     4  |  Siblings/Spouses Aboard  |      INTEGER  |
     5  |  Parents/Children Aboard  |      INTEGER  |
     6  |                     Fare  |       DOUBLE  |
*/

Table tm = Table.create()
    // .addColumns(IntColumn.create("Survived", Survived))
    .addColumns(IntColumn.create("Pclass", Pclass))
    // .addColumns(StringColumn.create("Name", Name))
    .addColumns(StringColumn.create("Sex", Sex))
    .addColumns(DoubleColumn.create("Age", Age))
    .addColumns(IntColumn.create("Siblings/Spouses Aboard", Siblings_Spouses_Aboard))
    .addColumns(IntColumn.create("Parents/Children Aboard", Parents_Children_Aboard))
    .addColumns(DoubleColumn.create("Fare", Fare));

// делаем те же самые преобразование с датасетом, что и при тренировке
tm.replaceColumn("Sex", tm.stringColumn("Sex").asDoubleColumn().asIntColumn());
tm;

// Делаем предсказание
int[] prediction = e.predict(tm.smile().toDataFrame());

int result = prediction[0];

result // 1 или 0

1